# Finding top 10 lowest RMSE in fitting vs experimental EIS data

In [7]:
# %% [markdown]
# Parsers and CSV reader for params_TL.csv → RMSE (5th column)

from pathlib import Path
import pandas as pd
import numpy as np
import re

MM_RE = re.compile(r"(\d+(?:\.\d+)?)\s*mM", re.IGNORECASE)
C_RE  = re.compile(r"(\d+(?:\.\d+)?)\s*°?\s*C", re.IGNORECASE)

def parse_conc_mm(path: Path):
    """Search upwards for a folder containing 'XmM' and return float X."""
    for p in [path] + list(path.parents):
        m = MM_RE.search(p.name)
        if m:
            return float(m.group(1))
    return None

def parse_temp_c(path: Path):
    """Search upwards for a folder containing 'YC' or 'Y°C' and return float Y."""
    for p in [path] + list(path.parents):
        m = C_RE.search(p.name)
        if m:
            return float(m.group(1))
    return None

def read_rmse_from_params(file: Path):
    """
    Read 'params_TL.csv' and return a single RMSE value from the 5th column (index 4).
    Tries header=None first; then tries with header inference.
    Returns float or None.
    """
    # Attempt 1: header=None
    try:
        df = pd.read_csv(file, header=None)
        if df.shape[1] >= 5:
            col = pd.to_numeric(df.iloc[:, 4], errors="coerce").dropna()
            if not col.empty:
                return float(col.iloc[0])
    except Exception:
        pass

    # Attempt 2: with header inference
    try:
        df = pd.read_csv(file)
        if df.shape[1] >= 5:
            col = pd.to_numeric(df.iloc[:, 4], errors="coerce").dropna()
            if not col.empty:
                return float(col.iloc[0])
    except Exception:
        pass

    return None


# %% Run the scan
ROOT = "/Users/hosseinostovar/Desktop/BACKUP/Data_H2SO4_NPG/data/NPG-500mM-H2SO4-whole_three"  


# --- FILTERS ---
# Option A: continuous range (inclusive). Set to None to disable.
conc_range = (15.0, 20.0)   # only 1–10 mM
# Option B: whitelist of specific concentrations (in mM). Set to [] to disable.
conc_list  = []            # e.g., [1, 2, 5, 10]

# Optional: also allow a temperature range filter if you want
temp_range = None          # e.g., (26.0, 40.0) or None to disable

# --- how many results to show ---
TOP_K = 10

from math import isfinite

def conc_ok(c):
    if c is None or not isfinite(c):
        return False
    ok = True
    if conc_range is not None:
        mn, mx = conc_range
        ok = ok and (mn <= c <= mx)
    if conc_list:
        # use a small tolerance to avoid FP gotchas
        ok = ok and any(abs(c - v) < 1e-9 for v in conc_list)
    return ok

def temp_ok(t):
    if t is None or not isfinite(t):
        return False
    if temp_range is None:
        return True
    mn, mx = temp_range
    return (mn <= t <= mx)

root = Path(ROOT)
records = []

for params_file in root.rglob("params_TL.csv"):
    conc = parse_conc_mm(params_file.parent)
    temp = parse_temp_c(params_file.parent)

    if not conc_ok(conc) or not temp_ok(temp):
        continue

    rmse = read_rmse_from_params(params_file)
    if rmse is None or not np.isfinite(rmse):
        continue

    records.append({
        "Concentration (mM)": conc,
        "Temperature (°C)": temp,
        "RMSE": float(rmse),
        "Path": str(params_file)
    })

if not records:
    raise RuntimeError("No valid entries found with the given filters. Check ROOT and filter settings.")

df = pd.DataFrame(records)

# If multiple files per (C,T), keep the lowest RMSE for that pair
agg = (df
       .sort_values("RMSE", ascending=True)
       .groupby(["Concentration (mM)", "Temperature (°C)"], as_index=False)
       .agg({"RMSE": "min"}))

topk = agg.sort_values("RMSE", ascending=True).head(TOP_K).reset_index(drop=True)
topk



,Concentration (mM),Temperature (°C),RMSE
0,15.0,40.0,0.003286
1,15.0,42.0,0.003297
2,15.0,44.0,0.003371
3,16.0,50.0,0.003431
4,15.0,46.0,0.003438
5,15.0,50.0,0.003441
6,18.0,50.0,0.003473
7,20.0,50.0,0.003483
8,19.0,50.0,0.003506
9,18.0,48.0,0.003518


# Pearson Correlation Plot

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# === CONFIG ===
# Path to your rounded EIS file (already used above)
CSV_PATH = "/Users/hosseinostovar/Desktop/BACKUP/Data_H2SO4_NPG/data/single_frequency/Single_frequencies_whole_spectrum/combined_eis.csv"

# Define frequency ranges (Hz) as a list of (low, high) tuples.
# Edit these as you like. Examples shown:
FREQ_RANGES = [
    (0.1, 1.0),
    (1.0, 100.0),
    (100.0, 1000.0),
    (1000.0, 10000.0),

]

# === LOAD & PREP ===
df = pd.read_csv(CSV_PATH)

# Map/standardize expected column names
df = df.rename(columns={
    "Frequency (Hz)": "frequency_Hz",
    "Z' (Ω)": "Z_real",
    "-Z'' (Ω)": "Z_imag_neg",
    "Concentration (mM)": "C_mM",
    "Temperature (°C)": "T_C"
})

needed = ["frequency_Hz","Z_real","Z_imag_neg","C_mM","T_C"]
missing = [c for c in needed if c not in df.columns]
if missing:
    raise ValueError(f"Missing columns in CSV: {missing}")

df = df[needed].dropna().reset_index(drop=True)

# === COMPUTE CORRELATIONS PER RANGE ===
rows = []
labels = []
for (lo, hi) in FREQ_RANGES:
    sub = df[(df["frequency_Hz"] >= float(lo)) & (df["frequency_Hz"] <= float(hi))].copy()
    label = f"[{lo:g}, {hi:g}]"
    labels.append(label)
    if len(sub) >= 3:  # need at least a few points for a meaningful r
        r_C_Zr = sub["C_mM"].corr(sub["Z_real"]) if sub["C_mM"].nunique() > 1 else np.nan
        r_C_Zi = sub["C_mM"].corr(sub["Z_imag_neg"]) if sub["C_mM"].nunique() > 1 else np.nan
        r_T_Zr = sub["T_C"].corr(sub["Z_real"])   if sub["T_C"].nunique() > 1 else np.nan
        r_T_Zi = sub["T_C"].corr(sub["Z_imag_neg"]) if sub["T_C"].nunique() > 1 else np.nan
    else:
        r_C_Zr = r_C_Zi = r_T_Zr = r_T_Zi = np.nan
    rows.append({
        "range": label,
        "r(C,Z')": r_C_Zr, "r(C,-Z'')": r_C_Zi,
        "r(T,Z')": r_T_Zr, "r(T,-Z'')": r_T_Zi,
        "N_points": len(sub)
    })

corr_by_range = pd.DataFrame(rows)

# Save table
out_csv = "/Users/hosseinostovar/Desktop/BACKUP/Data_H2SO4_NPG/data/single_frequency/Single_frequencies_whole_spectrum/plots/pearson_by_freq_range.csv"
corr_by_range.to_csv(out_csv, index=False)

# === PLOTS ===
# 1) Heatmap: rows = pairs, columns = ranges
pairs = ["r(C,Z')", "r(C,-Z'')", "r(T,Z')", "r(T,-Z'')"]
heat = corr_by_range[pairs].T.values  # shape 4 x n_ranges

fig1, ax1 = plt.subplots(figsize=(max(6, 1.2*len(FREQ_RANGES)), 3.5))
im = ax1.imshow(heat, vmin=-1, vmax=1, aspect="auto")
ax1.set_yticks(range(len(pairs))); ax1.set_yticklabels(pairs)
ax1.set_xticks(range(len(labels))); ax1.set_xticklabels(labels, rotation=45, ha="right")
for i in range(heat.shape[0]):
    for j in range(heat.shape[1]):
        val = heat[i, j]
        if np.isfinite(val):
            ax1.text(j, i, f"{val:.2f}", ha="center", va="center")
ax1.set_title("Pearson r by frequency range")
plt.tight_layout()
out_heat = "/Users/hosseinostovar/Desktop/BACKUP/Data_H2SO4_NPG/data/single_frequency/Single_frequencies_whole_spectrum/plots/pearson_heatmap_by_range.png"
plt.savefig(out_heat, dpi=180)
plt.close(fig1)

# 2) Line plot per pair vs. range index (x-axis labeled by range)
x = np.arange(len(labels))
fig2, ax2 = plt.subplots(figsize=(max(6, 1.2*len(FREQ_RANGES)), 4))
for p in pairs:
    ax2.plot(x, corr_by_range[p], label=p)
ax2.set_xticks(x); ax2.set_xticklabels(labels, rotation=45, ha="right")
ax2.set_ylim(-1, 1)
ax2.set_xlabel("Frequency range (Hz)")
ax2.set_ylabel("Pearson r")
ax2.set_title("Pearson r across frequency ranges")
ax2.legend()
plt.tight_layout()
out_lines = "/Users/hosseinostovar/Desktop/BACKUP/Data_H2SO4_NPG/data/single_frequency/Single_frequencies_whole_spectrum/plots/pearson_lines_by_range.png"
plt.savefig(out_lines, dpi=180)
plt.close(fig2)

out_csv, out_heat, out_lines


('/Users/hosseinostovar/Desktop/BACKUP/Data_H2SO4_NPG/data/single_frequency/Single_frequencies_whole_spectrum/plots/pearson_by_freq_range.csv',
 '/Users/hosseinostovar/Desktop/BACKUP/Data_H2SO4_NPG/data/single_frequency/Single_frequencies_whole_spectrum/plots/pearson_heatmap_by_range.png',
 '/Users/hosseinostovar/Desktop/BACKUP/Data_H2SO4_NPG/data/single_frequency/Single_frequencies_whole_spectrum/plots/pearson_lines_by_range.png')

# Jacobian sensitivity for C,T with respect to frequncy bands from PINN

In [8]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from pathlib import Path
from tqdm.auto import tqdm

# ============================
# CONFIG
# ============================
MODEL_PATH = "/Users/hosseinostovar/Desktop/BACKUP/Data_H2SO4_NPG/data/single_frequency/Single_frequencies_whole_spectrum/PINN_report/pinn_model.pt"       # path to your trained PINN
DATA_PATH  = "/Users/hosseinostovar/Desktop/BACKUP/Data_H2SO4_NPG/data/single_frequency/Single_frequencies_whole_spectrum/combined_eis_5-20mM.csv"     # path to your full impedance dataset
OUT_CSV    = "/Users/hosseinostovar/Desktop/BACKUP/Data_H2SO4_NPG/data/single_frequency/Single_frequencies_whole_spectrum/PINN_report/jacobian_band_information.csv"

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# Frequency bands (Hz)
BANDS = [
    {"name": "0_1",      "min": 0.2,    "max": 1.0},
    {"name": "1_100",    "min": 1.0,    "max": 100.0},
    {"name": "100_1k",   "min": 100.0,  "max": 1000.0},
    {"name": "1k_10k",   "min": 1000.0, "max": 10000.0},
]

# ============================
# Forward PINN definition (same as training)
# ============================
def _j_like(x):
    return torch.complex(torch.zeros((), dtype=x.dtype, device=x.device),
                         torch.ones( (), dtype=x.dtype, device=x.device))

def torch_coth(z, eps=1e-12):
    sz = torch.sinh(z); cz = torch.cosh(z)
    small = torch.abs(sz) < eps
    out = torch.empty_like(z)
    out[~small] = cz[~small] / sz[~small]
    out[small] = 1.0/z[small] + z[small]/3.0
    return out

def torch_zarc(Rp, Y0, n, w):
    j = _j_like(w)
    return 1.0 / (1.0/torch.clamp(Rp, min=1e-18) + torch.clamp(Y0, min=1e-18) * (j*w)**n)

def torch_tl_impedance(r, y0, n, L, w):
    j = _j_like(w)
    r_ = torch.clamp(r,  min=1e-18); y0_= torch.clamp(y0, min=1e-18)
    gamma = torch.sqrt(r_ * y0_ * (j*w)**n)
    Z0    = torch.sqrt(r_ / (y0_ * (j*w)**n))
    return Z0 * torch_coth(L * gamma)

def torch_impedance_rs_zarc_tl(omega, Rs, Rp, Y0, n0, r, y0, n1, L):
    Zarc = torch_zarc(Rp, Y0, n0, omega)
    Ztl  = torch_tl_impedance(r, y0, n1, L, omega)
    return Rs + Zarc + Ztl

class ThetaNet(nn.Module):
    def __init__(self, in_dim=2, width=64, depth=3, dtype=torch.float64):
        super().__init__()
        layers, d = [], in_dim
        for _ in range(depth):
            layers += [nn.Linear(d, width, dtype=dtype), nn.ReLU()]
            d = width
        self.backbone = nn.Sequential(*layers) if layers else nn.Identity()
        self.head = nn.Linear(d, 8, dtype=dtype)  # Rs, Rp, Y0, n0, r, y0, n1, L
        self.softplus = nn.Softplus(); self.sigmoid = nn.Sigmoid()
    def forward(self, Cn, Tn):
        h = self.backbone(torch.stack([Cn, Tn], dim=1))
        raw = self.head(h)
        Rs_r, Rp_r, Y0_r, n0_r, r_r, y0_r, n1_r, L_r = torch.unbind(raw, dim=1)
        eps = 1e-9
        Rs  = self.softplus(Rs_r)  + eps
        Rp  = self.softplus(Rp_r)  + eps
        Y0  = self.softplus(Y0_r)  + eps
        n0  = self.sigmoid(n0_r)
        r   = self.softplus(r_r)   + eps
        y0  = self.softplus(y0_r)  + eps
        n1  = self.sigmoid(n1_r)
        L   = self.softplus(L_r)   + eps
        return Rs, Rp, Y0, n0, r, y0, n1, L

class ForwardPINN:
    def __init__(self, model_path, device="cpu"):
        # Load checkpoint
        try:
            ckpt = torch.load(model_path, map_location=device, weights_only=False)
        except TypeError:
            ckpt = torch.load(model_path, map_location=device)

        self.xmu  = np.array(ckpt.get("xmu",  [0,0]), float)
        self.xstd = np.array(ckpt.get("xstd", [1,1]), float)

        tr = ckpt.get("train_config", {})
        width = int(tr.get("width", 64))
        depth = int(tr.get("depth", 3))

        self.net = ThetaNet(in_dim=2, width=width, depth=depth, dtype=torch.float64).to(device)
        self.net.load_state_dict(ckpt["state_dict"])
        self.net.eval()

        self.device = torch.device(device)
        self.y_norm = ckpt.get("y_norm", {"enabled": False})
        self._dtype = torch.float64

    def predict_torch(self, C_t, T_t, w_t):
        Cn = (C_t - float(self.xmu[0])) / (float(self.xstd[0]) + 1e-12)
        Tn = (T_t - float(self.xmu[1])) / (float(self.xstd[1]) + 1e-12)
        Rs,Rp,Y0,n0,r,y0,n1,L = self.net(Cn, Tn)
        Zc = torch_impedance_rs_zarc_tl(w_t, Rs,Rp,Y0,n0,r,y0,n1,L)
        y = torch.stack([Zc.real, -Zc.imag], dim=1)  # [Z', -Z'']

        yn = self.y_norm
        if yn.get("enabled", False):
            method = yn.get("method","standard")
            if method == "standard":
                mu  = torch.tensor(yn["mu"],  dtype=self._dtype, device=self.device)
                std = torch.tensor(yn["std"], dtype=self._dtype, device=self.device)
                y = y*std + mu
            elif method == "minmax":
                y_min = torch.tensor(yn["min"], dtype=self._dtype, device=self.device)
                y_max = torch.tensor(yn["max"], dtype=self._dtype, device=self.device)
                y = y*(y_max - y_min) + y_min
        return y

# ============================
# Load data, extract freqs and CT grid
# ============================
df = pd.read_csv(DATA_PATH)

df = df.rename(columns={
    "Frequency (Hz)": "frequency_Hz",
    "Concentration (mM)": "C_mM",
    "Temperature (°C)": "T_C",
})

needed = ["frequency_Hz", "C_mM", "T_C"]
missing = [c for c in needed if c not in df.columns]
if missing:
    raise ValueError(f"Missing columns in combined_eis.csv: {missing}")

df = df[needed].dropna().reset_index(drop=True)

freqs_all = np.sort(df["frequency_Hz"].astype(float).unique())
ct_pairs = df[["C_mM","T_C"]].drop_duplicates().to_numpy()
n_ct = ct_pairs.shape[0]

print(f"Found {len(freqs_all)} unique frequencies and {n_ct} unique (C,T) pairs.")

# ============================
# Jacobian-based info per band
# ============================
fwd = ForwardPINN(MODEL_PATH, device=DEVICE)

def band_freqs(freqs, band):
    lo, hi = band["min"], band["max"]
    return freqs[(freqs >= lo) & (freqs <= hi)]

def JtJ_for_CT_and_band(Cval, Tval, freqs_band):
    if freqs_band.size == 0:
        return np.zeros((2,2), float)

    device = fwd.device
    w = torch.tensor(2*np.pi*freqs_band, dtype=torch.float64, device=device)

    C = torch.tensor(float(Cval), dtype=torch.float64, device=device, requires_grad=True)
    T = torch.tensor(float(Tval), dtype=torch.float64, device=device, requires_grad=True)

    y = fwd.predict_torch(C.repeat(w.numel()), T.repeat(w.numel()), w)
    Zr = y[:,0]          # Z'
    Zim = -y[:,1]        # -Z''

    JtJ_sum = np.zeros((2,2), float)

    for i in range(len(freqs_band)):
        gC = torch.stack([
            torch.autograd.grad(Zr[i], C, retain_graph=True)[0],
            torch.autograd.grad(Zim[i], C, retain_graph=True)[0],
        ])
        gT = torch.stack([
            torch.autograd.grad(Zr[i], T, retain_graph=True)[0],
            torch.autograd.grad(Zim[i], T, retain_graph=True)[0],
        ])
        J = torch.stack([gC, gT], dim=1).detach().cpu().numpy()  # 2x2
        JtJ_sum += J.T @ J

    return JtJ_sum

band_info_sum = {b["name"]: np.zeros((2,2), float) for b in BANDS}
band_counts   = {b["name"]: 0 for b in BANDS}

print("\nComputing Jacobian-based information per band (this may take a bit)...")
for (Cval, Tval) in tqdm(ct_pairs, desc="(C,T) grid"):
    for b in BANDS:
        f_band = band_freqs(freqs_all, b)
        if f_band.size == 0:
            continue
        JtJ = JtJ_for_CT_and_band(Cval, Tval, f_band)
        band_info_sum[b["name"]] += JtJ
        band_counts[b["name"]]   += 1

rows = []
for b in BANDS:
    name = b["name"]
    if band_counts[name] == 0:
        continue
    I_avg = band_info_sum[name] / band_counts[name]
    I_CC = I_avg[0,0]
    I_TT = I_avg[1,1]
    I_CT = I_avg[0,1]
    det  = np.linalg.det(I_avg)
    tr   = np.trace(I_avg)
    ratio = I_CC / I_TT if I_TT > 0 else np.nan
    rows.append({
        "band": name,
        "f_min_Hz": b["min"],
        "f_max_Hz": b["max"],
        "I_CC": I_CC,
        "I_TT": I_TT,
        "I_CT": I_CT,
        "det_I": det,
        "trace_I": tr,
        "I_CC_over_I_TT": ratio,
        "N_CT": band_counts[name]
    })

band_df = pd.DataFrame(rows)
band_df.to_csv(OUT_CSV, index=False)
print("\nBand-wise Jacobian information:")
print(band_df)
print(f"\nSaved to {OUT_CSV}")



Found 100 unique frequencies and 208 unique (C,T) pairs.

Computing Jacobian-based information per band (this may take a bit)...


(C,T) grid:   0%|          | 0/208 [00:00<?, ?it/s]


Band-wise Jacobian information:
     band  f_min_Hz  f_max_Hz         I_CC       I_TT        I_CT  \
0     0_1       0.2       1.0  3389.299155  36.086935  272.680482   
1   1_100       1.0     100.0  7114.361166  51.008472  542.935842   
2  100_1k     100.0    1000.0  3155.831586  19.971963  224.442316   
3  1k_10k    1000.0   10000.0  1973.008837  13.126263  130.679527   

          det_I      trace_I  I_CC_over_I_TT  N_CT  
0  47954.772837  3425.386089       93.920394   208  
1  68113.360637  7165.369637      139.474110   208  
2  12653.799657  3175.803549      158.013087   208  
3   8821.093425  1986.135100      150.310023   208  

Saved to /Users/hosseinostovar/Desktop/BACKUP/Data_H2SO4_NPG/data/single_frequency/Single_frequencies_whole_spectrum/PINN_report/jacobian_band_information.csv


# Jacobian sensitivity for C,T with respect to frequncy bands AND Z', -Z''  from PINN

In [9]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from tqdm.auto import tqdm

# ============================
# CONFIG
# ============================
MODEL_PATH = "/Users/hosseinostovar/Desktop/BACKUP/Data_H2SO4_NPG/data/single_frequency/Single_frequencies_whole_spectrum/PINN_report/pinn_model.pt"       # path to your trained PINN
DATA_PATH  = "/Users/hosseinostovar/Desktop/BACKUP/Data_H2SO4_NPG/data/single_frequency/Single_frequencies_whole_spectrum/combined_eis_5-20mM.csv"     # full EIS dataset
OUT_CSV    = "/Users/hosseinostovar/Desktop/BACKUP/Data_H2SO4_NPG/data/single_frequency/Single_frequencies_whole_spectrum/PINN_report/jacobian_band_sensitivity_Zr_Zim.csv"

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

BANDS = [
    {"name": "0_1",      "min": 0.0,    "max": 1.0},
    {"name": "1_100",    "min": 1.0,    "max": 100.0},
    {"name": "100_1k",   "min": 100.0,  "max": 1000.0},
    {"name": "1k_10k",   "min": 1000.0, "max": 10000.0},
]

# ============================
# Forward PINN (same as before)
# ============================
def _j_like(x):
    return torch.complex(torch.zeros((), dtype=x.dtype, device=x.device),
                         torch.ones( (), dtype=x.dtype, device=x.device))

def torch_coth(z, eps=1e-12):
    sz = torch.sinh(z); cz = torch.cosh(z)
    small = torch.abs(sz) < eps
    out = torch.empty_like(z)
    out[~small] = cz[~small] / sz[~small]
    out[small] = 1.0/z[small] + z[small]/3.0
    return out

def torch_zarc(Rp, Y0, n, w):
    j = _j_like(w)
    return 1.0 / (1.0/torch.clamp(Rp, min=1e-18) + torch.clamp(Y0, min=1e-18) * (j*w)**n)

def torch_tl_impedance(r, y0, n, L, w):
    j = _j_like(w)
    r_ = torch.clamp(r,  min=1e-18); y0_= torch.clamp(y0, min=1e-18)
    gamma = torch.sqrt(r_ * y0_ * (j*w)**n)
    Z0    = torch.sqrt(r_ / (y0_ * (j*w)**n))
    return Z0 * torch_coth(L * gamma)

def torch_impedance_rs_zarc_tl(omega, Rs, Rp, Y0, n0, r, y0, n1, L):
    Zarc = torch_zarc(Rp, Y0, n0, omega)
    Ztl  = torch_tl_impedance(r, y0, n1, L, omega)
    return Rs + Zarc + Ztl

class ThetaNet(nn.Module):
    def __init__(self, in_dim=2, width=64, depth=3, dtype=torch.float64):
        super().__init__()
        layers, d = [], in_dim
        for _ in range(depth):
            layers += [nn.Linear(d, width, dtype=dtype), nn.ReLU()]
            d = width
        self.backbone = nn.Sequential(*layers) if layers else nn.Identity()
        self.head = nn.Linear(d, 8, dtype=dtype)  # Rs, Rp, Y0, n0, r, y0, n1, L
        self.softplus = nn.Softplus(); self.sigmoid = nn.Sigmoid()
    def forward(self, Cn, Tn):
        h = self.backbone(torch.stack([Cn, Tn], dim=1))
        raw = self.head(h)
        Rs_r, Rp_r, Y0_r, n0_r, r_r, y0_r, n1_r, L_r = torch.unbind(raw, dim=1)
        eps = 1e-9
        Rs  = self.softplus(Rs_r)  + eps
        Rp  = self.softplus(Rp_r)  + eps
        Y0  = self.softplus(Y0_r)  + eps
        n0  = self.sigmoid(n0_r)
        r   = self.softplus(r_r)   + eps
        y0  = self.softplus(y0_r)  + eps
        n1  = self.sigmoid(n1_r)
        L   = self.softplus(L_r)   + eps
        return Rs, Rp, Y0, n0, r, y0, n1, L

class ForwardPINN:
    def __init__(self, model_path, device="cpu"):
        try:
            ckpt = torch.load(model_path, map_location=device, weights_only=False)
        except TypeError:
            ckpt = torch.load(model_path, map_location=device)
        self.xmu  = np.array(ckpt.get("xmu",  [0,0]), float)
        self.xstd = np.array(ckpt.get("xstd", [1,1]), float)

        tr = ckpt.get("train_config", {})
        width = int(tr.get("width", 64))
        depth = int(tr.get("depth", 3))

        self.net = ThetaNet(in_dim=2, width=width, depth=depth, dtype=torch.float64).to(device)
        self.net.load_state_dict(ckpt["state_dict"])
        self.net.eval()

        self.device = torch.device(device)
        self.y_norm = ckpt.get("y_norm", {"enabled": False})
        self._dtype = torch.float64

    def predict_torch(self, C_t, T_t, w_t):
        Cn = (C_t - float(self.xmu[0])) / (float(self.xstd[0]) + 1e-12)
        Tn = (T_t - float(self.xmu[1])) / (float(self.xstd[1]) + 1e-12)
        Rs,Rp,Y0,n0,r,y0,n1,L = self.net(Cn, Tn)
        Zc = torch_impedance_rs_zarc_tl(w_t, Rs,Rp,Y0,n0,r,y0,n1,L)
        y = torch.stack([Zc.real, -Zc.imag], dim=1)  # [Z', -Z'']

        yn = self.y_norm
        if yn.get("enabled", False):
            method = yn.get("method","standard")
            if method == "standard":
                mu  = torch.tensor(yn["mu"],  dtype=self._dtype, device=self.device)
                std = torch.tensor(yn["std"], dtype=self._dtype, device=self.device)
                y = y*std + mu
            elif method == "minmax":
                y_min = torch.tensor(yn["min"], dtype=self._dtype, device=self.device)
                y_max = torch.tensor(yn["max"], dtype=self._dtype, device=self.device)
                y = y*(y_max - y_min) + y_min
        return y

# ============================
# Load data, get freqs and CT grid
# ============================
df_raw = pd.read_csv(DATA_PATH)
df = df_raw.rename(columns={
    "Frequency (Hz)": "frequency_Hz",
    "Concentration (mM)": "C_mM",
    "Temperature (°C)": "T_C",
})

needed = ["frequency_Hz", "C_mM", "T_C"]
missing = [c for c in needed if c not in df.columns]
if missing:
    raise ValueError(f"Missing columns in {DATA_PATH}: {missing}")

df = df[needed].dropna().reset_index(drop=True)

freqs_all = np.sort(df["frequency_Hz"].astype(float).unique())
ct_pairs = df[["C_mM","T_C"]].drop_duplicates().to_numpy()

print(f"Unique frequencies: {len(freqs_all)}, unique (C,T) pairs: {ct_pairs.shape[0]}")

# Precompute indices per band
indices_by_band = {}
for b in BANDS:
    mask = (freqs_all >= b["min"]) & (freqs_all <= b["max"])
    idx = np.where(mask)[0]
    indices_by_band[b["name"]] = idx

fwd = ForwardPINN(MODEL_PATH, device=DEVICE)

# ============================
# Accumulate squared sensitivities per band and output
# ============================
band_sums = {
    b["name"]: {
        "S_Zr_C": 0.0,
        "S_Zr_T": 0.0,
        "S_Zim_C": 0.0,
        "S_Zim_T": 0.0,
        "n_terms": 0
    }
    for b in BANDS
}

device = fwd.device
w_all = torch.tensor(2*np.pi*freqs_all, dtype=torch.float64, device=device)

print("\nComputing Z' / -Z'' sensitivities per band...")
for (Cval, Tval) in tqdm(ct_pairs, desc="(C,T) grid"):
    C = torch.tensor(float(Cval), dtype=torch.float64, device=device, requires_grad=True)
    T = torch.tensor(float(Tval), dtype=torch.float64, device=device, requires_grad=True)

    y = fwd.predict_torch(C.repeat(w_all.numel()), T.repeat(w_all.numel()), w_all)
    Zr_all = y[:,0]      # Z'
    Zim_all = -y[:,1]    # -Z''

    for band_name, idxs in indices_by_band.items():
        if idxs.size == 0:
            continue
        for i in idxs:
            Zr_i = Zr_all[i]
            Zi_i = Zim_all[i]

            gZr_C = torch.autograd.grad(Zr_i, C, retain_graph=True)[0]
            gZr_T = torch.autograd.grad(Zr_i, T, retain_graph=True)[0]
            gZi_C = torch.autograd.grad(Zi_i, C, retain_graph=True)[0]
            gZi_T = torch.autograd.grad(Zi_i, T, retain_graph=True)[0]

            bsum = band_sums[band_name]
            bsum["S_Zr_C"]  += float(gZr_C.detach().cpu().numpy()**2)
            bsum["S_Zr_T"]  += float(gZr_T.detach().cpu().numpy()**2)
            bsum["S_Zim_C"] += float(gZi_C.detach().cpu().numpy()**2)
            bsum["S_Zim_T"] += float(gZi_T.detach().cpu().numpy()**2)
            bsum["n_terms"] += 1

# Build summary DataFrame
rows = []
for b in BANDS:
    name = b["name"]
    s = band_sums[name]
    if s["n_terms"] == 0:
        continue
    n = s["n_terms"]
    rows.append({
        "band": name,
        "f_min_Hz": b["min"],
        "f_max_Hz": b["max"],
        "S_Zr_C_mean":   s["S_Zr_C"]  / n,
        "S_Zr_T_mean":   s["S_Zr_T"]  / n,
        "S_Zim_C_mean":  s["S_Zim_C"] / n,
        "S_Zim_T_mean":  s["S_Zim_T"] / n,
        # some useful ratios
        "ratio_Zr_vs_Zim_for_T":  (s["S_Zr_T"]  / s["S_Zim_T"]) if s["S_Zim_T"]  > 0 else np.nan,
        "ratio_Zr_vs_Zim_for_C":  (s["S_Zr_C"]  / s["S_Zim_C"]) if s["S_Zim_C"]  > 0 else np.nan,
        "ratio_T_vs_C_for_Zr":    (s["S_Zr_T"]  / s["S_Zr_C"])  if s["S_Zr_C"]  > 0 else np.nan,
        "ratio_T_vs_C_for_Zim":   (s["S_Zim_T"] / s["S_Zim_C"]) if s["S_Zim_C"] > 0 else np.nan,
        "n_terms": n,
    })

sens_df = pd.DataFrame(rows)
sens_df.to_csv(OUT_CSV, index=False)
print("\nSaved per-band Z'/ -Z'' sensitivities to:", OUT_CSV)
print(sens_df)


Unique frequencies: 100, unique (C,T) pairs: 208

Computing Z' / -Z'' sensitivities per band...


(C,T) grid:   0%|          | 0/208 [00:00<?, ?it/s]


Saved per-band Z'/ -Z'' sensitivities to: /Users/hosseinostovar/Desktop/BACKUP/Data_H2SO4_NPG/data/single_frequency/Single_frequencies_whole_spectrum/PINN_report/jacobian_band_sensitivity_Zr_Zim.csv
     band  f_min_Hz  f_max_Hz  S_Zr_C_mean  S_Zr_T_mean  S_Zim_C_mean  \
0     0_1       0.0       1.0   238.254679     2.148751      3.838118   
1   1_100       1.0     100.0   191.673159     1.332645      0.606872   
2  100_1k     100.0    1000.0   164.244758     1.030300      1.851641   
3  1k_10k    1000.0   10000.0   105.679849     0.700972      3.931753   

   S_Zim_T_mean  ratio_Zr_vs_Zim_for_T  ratio_Zr_vs_Zim_for_C  \
0      0.428887               5.010061              62.075914   
1      0.045963              28.994068             315.837663   
2      0.020856              49.401378              88.702278   
3      0.028265              24.799725              26.878554   

   ratio_T_vs_C_for_Zr  ratio_T_vs_C_for_Zim  n_terms  
0             0.009019              0.111744     291

# 1. Band-wise percentage contributions (“sensitivity shares”) 



In [ ]:
import pandas as pd
import numpy as np

# === INPUT / OUTPUT ===
SENS_PATH = "/Users/hosseinostovar/Desktop/BACKUP/Data_H2SO4_NPG/data/single_frequency/Single_frequencies_whole_spectrum/PINN_report/sensitivity_analysis/jacobian_band_sensitivity_Zr_Zim.csv"
OUT_CSV   = "/Users/hosseinostovar/Desktop/BACKUP/Data_H2SO4_NPG/data/single_frequency/Single_frequencies_whole_spectrum/PINN_report/band_sensitivity_decomposed.csv"

# Load the bandwise derivative stats
sens = pd.read_csv(SENS_PATH)

# We expect columns like:
# band, f_min_Hz, f_max_Hz, S_Zr_C_mean, S_Zr_T_mean, S_Zim_C_mean, S_Zim_T_mean, n_terms, ...

# 1) Compute total (integrated) squared sensitivities per band
for part, param in [("Zr","C"), ("Zr","T"), ("Zim","C"), ("Zim","T")]:
    mean_col = f"S_{part}_{param}_mean"
    total_col = f"S_{part}_{param}_total"
    sens[total_col] = sens[mean_col] * sens["n_terms"]

# Total Zr and Zim sensitivity in each band (C + T together)
sens["S_Zr_total"]  = sens["S_Zr_C_total"]  + sens["S_Zr_T_total"]
sens["S_Zim_total"] = sens["S_Zim_C_total"] + sens["S_Zim_T_total"]

# Global totals over all bands
Zr_total_all  = sens["S_Zr_total"].sum()
Zim_total_all = sens["S_Zim_total"].sum()

# 2) Band shares of overall Zr / Zim sensitivity (C+T combined)
sens["share_Zr_band"]  = sens["S_Zr_total"]  / Zr_total_all         # sums to 1 over bands
sens["share_Zim_band"] = sens["S_Zim_total"] / Zim_total_all        # sums to 1 over bands

# 3) Within each band: fraction of Zr/Zim sensitivity due to C vs T
sens["frac_Zr_C_in_band"]   = sens["S_Zr_C_total"]   / sens["S_Zr_total"]
sens["frac_Zr_T_in_band"]   = sens["S_Zr_T_total"]   / sens["S_Zr_total"]
sens["frac_Zim_C_in_band"]  = sens["S_Zim_C_total"]  / sens["S_Zim_total"]
sens["frac_Zim_T_in_band"]  = sens["S_Zim_T_total"]  / sens["S_Zim_total"]

# 4) Global contributions of (C,T) for Z′ and −Z″ *with band weight*
#    These partition the *entire* Zr / Zim sensitivity budget.
#    For Zr: sum_b (global_Zr_C_share_band + global_Zr_T_share_band) = 1
sens["global_Zr_C_share"]   = sens["S_Zr_C_total"]   / Zr_total_all
sens["global_Zr_T_share"]   = sens["S_Zr_T_total"]   / Zr_total_all
sens["global_Zim_C_share"]  = sens["S_Zim_C_total"]  / Zim_total_all
sens["global_Zim_T_share"]  = sens["S_Zim_T_total"]  / Zim_total_all

# 5) Build a clean output table with both fractions and percentages
out = sens[[
    "band","f_min_Hz","f_max_Hz",
    # band-level shares (C+T combined)
    "share_Zr_band","share_Zim_band",
    # within-band C vs T for Zr / Zim
    "frac_Zr_C_in_band","frac_Zr_T_in_band",
    "frac_Zim_C_in_band","frac_Zim_T_in_band",
    # global C/T contributions with band weighting
    "global_Zr_C_share","global_Zr_T_share",
    "global_Zim_C_share","global_Zim_T_share",
]].copy()

# Also make a percentage version for nicer reading
out_pct = out.copy()
for col in out_pct.columns:
    if col not in ["band","f_min_Hz","f_max_Hz"]:
        out_pct[col + "_pct"] = 100.0 * out_pct[col]

out_pct.to_csv(OUT_CSV, index=False)
print("Saved decomposed band sensitivities to:", OUT_CSV)
print(out_pct)


| Band     | Z′ share over bands | −Z″ share over bands | In band: Z′ from C | Z′ from T | In band: −Z″ from C | −Z″ from T | Global: Z′ from C in this band | Global: Z′ from T in this band | Global: −Z″ from C in this band | Global: −Z″ from T in this band |
| -------- | ------------------- | -------------------- | ------------------ | --------- | ------------------- | ---------- | ------------------------------ | ------------------------------ | ------------------------------- | ------------------------------- |
| 0–1 Hz   | 21.6%               | 31.3%                | 99.1%              | 0.9%      | 89.9%               | 10.1%      | 21.4%                          | 0.19%                          | 28.2%                           | 3.15%                           |
| 1–100 Hz | 45.9%               | 12.7%                | 99.3%              | 0.7%      | 93.0%               | 7.0%       | 45.6%                          | 0.32%                          | 11.8%                           | 0.89%                           |
| 100–1k   | 20.2%               | 18.7%                | 99.4%              | 0.6%      | 98.9%               | 1.1%       | 20.1%                          | 0.13%                          | 18.4%                           | 0.21%                           |
| 1k–10k   | 12.3%               | 37.4%                | 99.3%              | 0.7%      | 99.3%               | 0.7%       | 12.2%                          | 0.08%                          | 37.1%                           | 0.27%                           |


# 2. Dimensionless “relative sensitivity” using C,T ranges

In [11]:
import pandas as pd
import numpy as np

# === INPUT ===
SENS_PATH = "/Users/hosseinostovar/Desktop/BACKUP/Data_H2SO4_NPG/data/single_frequency/Single_frequencies_whole_spectrum/PINN_report/sensitivity_analysis/jacobian_band_sensitivity_Zr_Zim.csv"
EIS_PATH  = "/Users/hosseinostovar/Desktop/BACKUP/Data_H2SO4_NPG/data/single_frequency/Single_frequencies_whole_spectrum/combined_eis_5-20mM.csv"
OUT_DIMLESS_CSV = "/Users/hosseinostovar/Desktop/BACKUP/Data_H2SO4_NPG/data/single_frequency/Single_frequencies_whole_spectrum/PINN_report/sensitivity_analysis/band_dimensionless_sensitivity.csv"

# Load sensitivities
sens = pd.read_csv(SENS_PATH)

# Load raw EIS data and standardize column names
eis = pd.read_csv(EIS_PATH)
eis = eis.rename(columns={
    "Frequency (Hz)": "frequency_Hz",
    "Z' (Ω)":         "Zr",
    "-Z'' (Ω)":       "Zim",
    "Concentration (mM)": "C_mM",
    "Temperature (°C)":   "T_C",
})

needed = ["frequency_Hz","Zr","Zim","C_mM","T_C"]
missing = [c for c in needed if c not in eis.columns]
if missing:
    raise ValueError(f"Missing columns in {EIS_PATH}: {missing}")

eis = eis[needed].dropna().reset_index(drop=True)

# Determine parameter ranges from data
dC = eis["C_mM"].max() - eis["C_mM"].min()  # ΔC
dT = eis["T_C"].max() - eis["T_C"].min()    # ΔT

print(f"ΔC (mM) from data: {dC:.3f}")
print(f"ΔT (°C) from data: {dT:.3f}")

# Compute RMS of Zr and Zim in each band
rms_rows = []
for _, row in sens.iterrows():
    band = row["band"]
    fmin = row["f_min_Hz"]
    fmax = row["f_max_Hz"]
    sub = eis[(eis["frequency_Hz"] >= fmin) & (eis["frequency_Hz"] <= fmax)]
    if sub.empty:
        Zr_rms = np.nan
        Zim_rms = np.nan
    else:
        Zr_rms = float(np.sqrt((sub["Zr"]**2).mean()))
        Zim_rms = float(np.sqrt((sub["Zim"]**2).mean()))
    rms_rows.append({"band": band, "Zr_rms": Zr_rms, "Zim_rms": Zim_rms})

rms_df = pd.DataFrame(rms_rows)

# Merge RMS back into sensitivity table
sens2 = sens.merge(rms_df, on="band", how="left")

# Compute dimensionless sensitivities:
#   tilde_S_Zr_C  = S_Zr_C_mean  * (ΔC / Zr_rms)^2
#   tilde_S_Zr_T  = S_Zr_T_mean  * (ΔT / Zr_rms)^2
#   tilde_S_Zim_C = S_Zim_C_mean * (ΔC / Zim_rms)^2
#   tilde_S_Zim_T = S_Zim_T_mean * (ΔT / Zim_rms)^2

sens2["tilde_S_Zr_C"]  = sens2["S_Zr_C_mean"]  * (dC**2 / sens2["Zr_rms"]**2)
sens2["tilde_S_Zr_T"]  = sens2["S_Zr_T_mean"]  * (dT**2 / sens2["Zr_rms"]**2)
sens2["tilde_S_Zim_C"] = sens2["S_Zim_C_mean"] * (dC**2 / sens2["Zim_rms"]**2)
sens2["tilde_S_Zim_T"] = sens2["S_Zim_T_mean"] * (dT**2 / sens2["Zim_rms"]**2)

# Build compact output table
out_cols = [
    "band","f_min_Hz","f_max_Hz",
    "Zr_rms","Zim_rms",
    "tilde_S_Zr_C","tilde_S_Zr_T",
    "tilde_S_Zim_C","tilde_S_Zim_T",
]
dimless = sens2[out_cols].copy()

dimless.to_csv(OUT_DIMLESS_CSV, index=False)
print("Saved dimensionless band sensitivities to:", OUT_DIMLESS_CSV)
print(dimless)


ΔC (mM) from data: 15.000
ΔT (°C) from data: 24.000
Saved dimensionless band sensitivities to: /Users/hosseinostovar/Desktop/BACKUP/Data_H2SO4_NPG/data/single_frequency/Single_frequencies_whole_spectrum/PINN_report/sensitivity_analysis/band_dimensionless_sensitivity.csv
     band  f_min_Hz  f_max_Hz      Zr_rms    Zim_rms  tilde_S_Zr_C  \
0     0_1       0.0       1.0  144.729695  34.143789      2.559226   
1   1_100       1.0     100.0  125.459403  10.213741      2.739917   
2  100_1k     100.0    1000.0  111.256271  10.318507      2.985555   
3  1k_10k    1000.0   10000.0   90.256679  14.919071      2.918878   

   tilde_S_Zr_T  tilde_S_Zim_C  tilde_S_Zim_T  
0      0.059087       0.740760       0.211905  
1      0.048767       1.308911       0.253780  
2      0.047944       3.912961       0.112827  
3      0.049564       3.974525       0.073146  
